In [39]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import avg, sum, col,lit
import configparser
import pandas as pd

parser = configparser.ConfigParser()
parser.read("config.yml")
user = parser.get("Credentials", "username")
password = parser.get("Credentials", "password")
acctName = parser.get("Credentials", "account")
wh = parser.get("Credentials", "warehouse")
dbname = parser.get("Credentials", "database")
schema = parser.get("Credentials", "schema")
role = parser.get("Credentials", "role")

In [41]:
#bgouoha-kn32396
connection_parameters = {
   "account": acctName,
   "user": user,
   "password": password,
   "warehouse": wh,
   "role": role,
   "database": "TEST_DB",
   "schema": "TEST_SCHEMA"
}
session = Session.builder.configs(connection_parameters).create()


# test if we have a connection
session.sql("select current_warehouse() wh, current_database() db, current_schema() schema, current_version() v,current_role() role").show()

--------------------------------------------------------------
|"WH"        |"DB"     |"SCHEMA"     |"V"     |"ROLE"        |
--------------------------------------------------------------
|COMPUTE_WH  |TEST_DB  |TEST_SCHEMA  |7.17.0  |ACCOUNTADMIN  |
--------------------------------------------------------------



In [42]:
df1 = session.create_dataframe([1, 2, 3, 4]).to_df("a") 
df1.show()

-------
|"A"  |
-------
|1    |
|2    |
|3    |
|4    |
-------



In [43]:
df2 = session.create_dataframe([[1, 2, 3, 4],[5,6,7,8]], schema=["a", "b", "c", "d"])
df2.show()

-------------------------
|"A"  |"B"  |"C"  |"D"  |
-------------------------
|1    |2    |3    |4    |
|5    |6    |7    |8    |
-------------------------



In [45]:
from snowflake.snowpark import Row
df3 = session.create_dataframe([Row(a=1, b=2, c=3, d=4)])
df3.show()

-------------------------
|"A"  |"B"  |"C"  |"D"  |
-------------------------
|1    |2    |3    |4    |
-------------------------



In [16]:
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField
schema = StructType([StructField("a", IntegerType()), StructField("b", StringType())])
df4 = session.create_dataframe([[1, "snow"], [3, "flake"]], schema)
df4.show()

---------------
|"A"  |"B"    |
---------------
|1    |snow   |
|3    |flake  |
---------------



In [46]:
df_range = session.range(1, 10, 2).to_df("a")
df_range.show()

-------
|"A"  |
-------
|1    |
|3    |
|5    |
|7    |
|9    |
-------



In [48]:
session.file.put("Student_Info.csv", "@my_internal_stage/Student_Info/")

[PutResult(source='Student_Info.csv', target='Student_Info.csv.gz', source_size=156, target_size=176, source_compression='NONE', target_compression='GZIP', status='UPLOADED', message='')]

In [49]:
from snowflake.snowpark.types import StructType, StructField, StringType, IntegerType, FloatType

# Create DataFrames from data in a stage.
df_catalog = session.read.schema(StructType([StructField("id", StringType()), StructField("name", StringType()),StructField("percentage", StringType())])).csv("@my_internal_stage/Sample_Files/")
df_catalog.show()

---------------------------------
|"ID"  |"NAME"   |"PERCENTAGE"  |
---------------------------------
|ID    |Name     |Percentage    |
|1     |Harsha   |87.5          |
|2     |Roy      |67            |
|3     |Ram      |97.4          |
|4     |Joy      |39.5          |
|5     |Francis  |57            |
|6     |Raksha   |76            |
|7     |Darshan  |36            |
|8     |Garry    |89.3          |
|9     |Jose     |45.2          |
---------------------------------



In [51]:
from snowflake.snowpark.functions import col

df_product_info = df_catalog
df1 = df_product_info.select(df_product_info.id, df_product_info["percentage"])
df1.show()


-----------------------
|"ID"  |"PERCENTAGE"  |
-----------------------
|ID    |Percentage    |
|1     |87.5          |
|2     |67            |
|3     |97.4          |
|4     |39.5          |
|5     |57            |
|6     |76            |
|7     |36            |
|8     |89.3          |
|9     |45.2          |
-----------------------



In [52]:
df2 = df_product_info.select(df_product_info.id, df_product_info.name, df_product_info.percentage)
df2.show()

---------------------------------
|"ID"  |"NAME"   |"PERCENTAGE"  |
---------------------------------
|ID    |Name     |Percentage    |
|1     |Harsha   |87.5          |
|2     |Roy      |67            |
|3     |Ram      |97.4          |
|4     |Joy      |39.5          |
|5     |Francis  |57            |
|6     |Raksha   |76            |
|7     |Darshan  |36            |
|8     |Garry    |89.3          |
|9     |Jose     |45.2          |
---------------------------------



In [53]:
df3 = df_product_info.select("id", "name", "percentage")
df3.show()

---------------------------------
|"ID"  |"NAME"   |"PERCENTAGE"  |
---------------------------------
|ID    |Name     |Percentage    |
|1     |Harsha   |87.5          |
|2     |Roy      |67            |
|3     |Ram      |97.4          |
|4     |Joy      |39.5          |
|5     |Francis  |57            |
|6     |Raksha   |76            |
|7     |Darshan  |36            |
|8     |Garry    |89.3          |
|9     |Jose     |45.2          |
---------------------------------



In [54]:
df_lhs = session.create_dataframe([["a", 1], ["b", 2]], schema=["key", "value1"])
df_rhs = session.create_dataframe([["a", 3], ["b", 4]], schema=["key", "value2"])
df_lhs.show()
df_rhs.show()

--------------------
|"KEY"  |"VALUE1"  |
--------------------
|a      |1         |
|b      |2         |
--------------------

--------------------
|"KEY"  |"VALUE2"  |
--------------------
|a      |3         |
|b      |4         |
--------------------



In [55]:
df_lhs.join(df_rhs, df_lhs.col("key") == df_rhs.col("key")).select(df_lhs["key"].as_("key"), "value1", "value2").show()

-------------------------------
|"KEY"  |"VALUE1"  |"VALUE2"  |
-------------------------------
|a      |1         |3         |
|b      |2         |4         |
-------------------------------



In [56]:
from snowflake.snowpark.exceptions import SnowparkJoinException
df=session.table("locally_loaded_table")
try:
    df_joined = df.join(df, col("id") == col("parent_id")) # fails
except SnowparkJoinException as e:
    print(e.message)

You cannot join a DataFrame with itself because the column references cannot be resolved correctly. Instead, create a copy of the DataFrame with copy.copy(), and join the DataFrame with this copy.


In [57]:
from copy import copy
df_lhs = session.create_dataframe([["a", 1], ["b", 2]], schema=["key", "value1"])
df_rhs = session.create_dataframe([["a", 3], ["b", 4]], schema=["key", "value2"])
df_lhs_copied = copy(df_lhs)
df_self_joined = df_lhs.join(df_lhs_copied, (df_lhs.col("key") == df_lhs_copied.col("key")) & (df_lhs.col("value1") == df_lhs_copied.col("value1")))
df_self_joined.show()

-------------------------------------------------------------------
|"l_k8ne_KEY"  |"l_k8ne_VALUE1"  |"r_58fs_KEY"  |"r_58fs_VALUE1"  |
-------------------------------------------------------------------
|a             |1                |a             |1                |
|b             |2                |b             |2                |
-------------------------------------------------------------------



In [58]:
>>> df = session.create_dataframe([[1, 3], [2, 10]], schema=["a", "b"])
>>> df.show()
>>> # Specify the equivalent of "WHERE a + b < 10"
>>> # in a SQL SELECT statement.
>>> df_filtered = df.filter((col("a") + col("b")) < 10)
>>> df_filtered.show()
>>> # To return the DataFrame as a table in a Python worksheet use return instead of show()
>>> # return df_filtered

-------------
|"A"  |"B"  |
-------------
|1    |3    |
|2    |10   |
-------------

-------------
|"A"  |"B"  |
-------------
|1    |3    |
-------------



In [59]:
>>> from snowflake.snowpark.exceptions import SnowparkSQLException
>>> try:
...     df.select(col("ID")).collect()
... except SnowparkSQLException as e:
...     print(e.message)

000904 (42000): SQL compilation error: error line 1 at position 7
invalid identifier 'ID'


In [60]:
df.write.mode("overwrite").save_as_table("table1")

In [22]:
df_table=session.table("table1")
df.show()

-------------
|"A"  |"B"  |
-------------
|1    |3    |
|2    |10   |
-------------



In [61]:
df.create_or_replace_view("Sample_View")

[Row(status='View SAMPLE_VIEW successfully created.')]

In [25]:
df_view=session.table("Sample_View")
df.show()

-------------
|"A"  |"B"  |
-------------
|1    |3    |
|2    |10   |
-------------



In [63]:
>>> # Import the upper function from the functions module.
>>> from snowflake.snowpark.functions import upper, col
>>> session.table("local_loaded_table").show()
>>> session.table("local_loaded_table").select(upper(col("name")).alias("upper_name")).collect()

-----------------------------------------------
|"ID"  |"NAME"   |"DESIGNATION"   |"CITY"     |
-----------------------------------------------
|20    |Ajay     |Data Analyst 1  |Noida      |
|17    |Vihay    |DataData        |Bengaluru  |
|20    |Ajay     |Data Analyst 1  |Noida      |
|21    |Vihay    |DataData        |Bengaluru  |
|12    |Nobody   |Data Analyst 1  |Noida      |
|13    |Chinmay  |DataData        |BBB        |
|13    |Karthik  |Data Analyst 1  |Karkala    |
-----------------------------------------------



[Row(UPPER_NAME='AJAY'),
 Row(UPPER_NAME='VIHAY'),
 Row(UPPER_NAME='NOBODY'),
 Row(UPPER_NAME='CHINMAY'),
 Row(UPPER_NAME='KARTHIK'),
 Row(UPPER_NAME='AJAY'),
 Row(UPPER_NAME='VIHAY')]

In [64]:
>>> # Import the call_function function from the functions module.
>>> from snowflake.snowpark.functions import call_function
>>> df = session.create_dataframe([[60, 2], [180, 4]], schema=["col1", "col2"])
>>> # Call the system-defined function RADIANS() on col1.
>>> df_result=df.select(call_function("radians", col("col1"))).collect()
>>> print(df_result)

[Row(RADIANS("COL1")=1.047197551), Row(RADIANS("COL1")=3.141592654)]
